In [3]:
#import
import numpy as np

import torch
from torch.autograd import Variable
from torch import optim

In [4]:
#load in processed data from CollegeBasketballPredictor.ipynb
def load_marchmadness():
    #features from training, test, validation sets
    featurestrainingData = torch.load('featurestraining_pytorch.pt')
    featurestestData = torch.load('featurestest_pytorch.pt')
    featuresvalidationData = torch.load('featuresvalidation_pytorch.pt')
    
    #labels from training, test, validation sets
    labelstrainingData = torch.load('labelstraining_pytorch.pt')
    labelstestData = torch.load('labelstest_pytorch.pt')
    labelsvalidationData = torch.load('labelsvalidation_pytorch.pt')
    
    return featurestrainingData, labelstrainingData, featurestestData, labelstestData, featuresvalidationData, labelsvalidationData

In [5]:
def build_model(input_dim, output_dim):
    # We don't need the softmax layer here since CrossEntropyLoss already
    # uses it internally.
    model = torch.nn.Sequential()
    model.add_module("linear",
                     torch.nn.Linear(input_dim, output_dim, bias=False))
    return model


In [6]:
def train(model, loss, optimizer, x, y):
    # Reset gradient
    optimizer.zero_grad()

    # Forward
    fx = model.forward(x)
    output = loss.forward(fx, y)

    # Backward
    output.backward()

    # Update parameters
    optimizer.step()

    return output.item()


In [7]:
def predict(model, x):
    output = model.forward(x)
    return output.data.numpy().argmax(axis=1)

In [10]:
def main():
    torch.manual_seed(42)
    #X is data Y is label
    trX, trY, teX, teY, valX, valY = load_marchmadness()
    
    n_examples, n_features = list(trX.size())[0], list(trX.size())[1]
    n_classes = 2
    model = build_model(n_features, n_classes)
    loss = torch.nn.CrossEntropyLoss(reduction='elementwise_mean')
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    batch_size = 100

    for i in range(100):
        cost = 0.
        num_batches = n_examples // batch_size
        for k in range(num_batches):
            start, end = k * batch_size, (k + 1) * batch_size
            cost += train(model, loss, optimizer,
                          trX[start:end], trY[start:end])
        predY = predict(model, teX)
        print("Epoch %d, cost = %f, acc = %.2f%%"
              #.cpu().numpy() turns it back into a numpy array
              % (i + 1, cost / num_batches, 100. * np.mean(predY == teY.cpu().numpy())))


In [11]:
main()

Epoch 1, cost = 16267.832752, acc = 50.61%
Epoch 2, cost = 10987.711009, acc = 49.39%
Epoch 3, cost = 4924.188942, acc = 87.20%
Epoch 4, cost = 1357.066021, acc = 51.23%
Epoch 5, cost = 1207.638172, acc = 83.53%
Epoch 6, cost = 818.866871, acc = 50.83%
Epoch 7, cost = 1503.400202, acc = 50.63%
Epoch 8, cost = 750.071317, acc = 71.11%
Epoch 9, cost = 977.262725, acc = 79.18%
Epoch 10, cost = 940.798901, acc = 54.62%
Epoch 11, cost = 524.938507, acc = 89.57%
Epoch 12, cost = 599.135572, acc = 56.13%
Epoch 13, cost = 559.270000, acc = 77.39%
Epoch 14, cost = 516.386148, acc = 72.83%
Epoch 15, cost = 390.007066, acc = 94.27%
Epoch 16, cost = 498.449073, acc = 74.39%
Epoch 17, cost = 590.853183, acc = 74.45%
Epoch 18, cost = 509.761003, acc = 88.21%
Epoch 19, cost = 410.626250, acc = 74.41%
Epoch 20, cost = 444.118327, acc = 87.51%
Epoch 21, cost = 364.283712, acc = 87.73%
Epoch 22, cost = 409.625442, acc = 74.80%
Epoch 23, cost = 490.492705, acc = 95.12%
Epoch 24, cost = 364.481725, acc = 